In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import pandas as pd
from preprocessing import Preprocessing
import numpy as np
import tfidf
from sklearn.cluster import Birch

prp = Preprocessing()
df = prp.main()
df = Preprocessing.preprocess(df)
df = df['text']
ntf = tfidf.TFIDF(df)
X = ntf.df_tfidf_vectorize(df)

brc = Birch(branching_factor=400, n_clusters=4000, threshold=0.5).fit(X)


starting preprocessing...
...preprocessing completed
starting vectorizing words...
...words vectorized


In [10]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=100)
vectoriser.fit_transform(df)
result = pd.concat([df,pd.DataFrame(X.toarray(),columns=vectoriser.get_feature_names())],axis=1)
result['cluster'] = brc.predict(X)

/home/annabelle/Documents/TwiiterSentimentAnalysis/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
result[['text', 'cluster']]

,text,cluster
0,in n out equal gross hah i onlli eat that afte...,181
1,ive just realiz why my finger hurt its because...,462
2,final newspapergo to print in the morn,660
3,doing my nail x,927
4,is off to the lake then work again,2145
...,...,...
89995,dang it when did the giant get the rocket as m...,1101
89996,got to work when its sunni out,3311
89997,rememb your mother today yes she might be anno...,250
89998,i would have settl for usd,1814


In [12]:
clusters = result['cluster'].unique()
labels = []
for i in range(len(clusters)):
    subset = result[result['cluster'] == clusters[i]]
    words = ' '.join([x for x in np.where(subset.all()!=0,subset.columns,None) if x and x!='text' and x!='cluster' and len(x.split()) == 1])
    labels.append(words)
labels_table = pd.DataFrame(zip(clusters,labels),columns=['cluster','label'])
result_labelled = pd.merge(result,labels_table,on='cluster',how='left')

In [13]:
result_labelled[['text', 'cluster', 'label']]

,text,cluster,label
0,in n out equal gross hah i onlli eat that afte...,181,out
1,ive just realiz why my finger hurt its because...,462,its too
2,final newspapergo to print in the morn,660,in the
3,doing my nail x,927,my
4,is off to the lake then work again,2145,off then
...,...,...,...
89995,dang it when did the giant get the rocket as m...,1101,as
89996,got to work when its sunni out,3311,out when
89997,rememb your mother today yes she might be anno...,250,be your
89998,i would have settl for usd,1814,for


In [14]:
pd.pivot_table(result_labelled, index=['label'], values=['text'], aggfunc='count').sort_values(['text'], ascending=False).rename(columns={'text':'count of text'})

,count of text
label,
,8936
go to,1389
the,1023
for the,863
in the,856
...,...
as day,4
if is there,4
dont thank,4


In [15]:
sil_score = silhouette_score(X, brc.labels_)
sil_score

0.12218790215750593

In [8]:
from sklearn.metrics import davies_bouldin_score
davies_bouldin_score(X.toarray(), brc.labels_)

2.200387964496554

0.09
3.46